In [23]:
# Submission for task-2 Abhay Bedi (Data Scientist Intern) Thapar Institute of Engineering & Technology

import pandas as pd
from sqlalchemy import create_engine


# Read the CSV or Excel file
file_path = '/content/used_bikes.csv'  # replace with the actual file path
data = pd.read_csv(file_path)  # use pd.read_excel() if the file is in Excel format

data


,bike_name,price,city,kms_driven,owner,age,power,brand
0,TVS Star City Plus Dual Tone 110cc,35000.0,Ahmedabad,17654.0,First Owner,3.0,110.0,TVS
1,Royal Enfield Classic 350cc,119900.0,Delhi,11000.0,First Owner,4.0,350.0,Royal Enfield
2,Triumph Daytona 675R,600000.0,Delhi,110.0,First Owner,8.0,675.0,Triumph
3,TVS Apache RTR 180cc,65000.0,Bangalore,16329.0,First Owner,4.0,180.0,TVS
4,Yamaha FZ S V 2.0 150cc-Ltd. Edition,80000.0,Bangalore,10000.0,First Owner,3.0,150.0,Yamaha
...,...,...,...,...,...,...,...,...
144,Royal Enfield Standard 350cc,115000.0,Hyderabad,14900.0,First Owner,3.0,350.0,Royal Enfield
145,Honda CB Shine 125cc Disc,65000.0,Mumbai,2000.0,First Owner,3.0,125.0,Honda
146,Honda CB Unicorn ABS 150cc,99000.0,Mumbai,4000.0,First Owner,2.0,150.0,Honda
147,Yamaha YZF-R15 2.0 150cc,68500.0,Delhi,68500.0,Second Owner,7.0,150.0,Yamaha


In [25]:
# Check memory usage before changing data types
print("Memory Usage Before:")
print(data.memory_usage(deep=True))

# Changing data types of columns as needed
#(for columns containing int/float values astype function can simply be applied)
data['bike_name'] = pd.to_numeric(data['bike_name'], errors='coerce').astype('Int16')
data['price'] = data['price'].astype(int)
data['kms_driven'] = data['kms_driven'].astype(int)
data['age'] = data['age'].astype(int)
data['power'] = data['power'].astype(int)
data['city'] = pd.to_numeric(data['city'], errors='coerce').astype('Int16')
data['owner'] = pd.to_numeric(data['owner'], errors='coerce').astype('Int16')
data['brand'] = pd.to_numeric(data['brand'], errors='coerce').astype('Int16')
# (for columns containg strings) pd.to_numeric function is used to convert the values in the 'Owner Name' column to numeric values,
# and the astype('Int64') method is then used to convert them to integers. The errors='coerce' parameter ensures that non-convertible values are set to NaN.



# Check memory usage after changing data types
print("\nMemory Usage After:")
print(data.memory_usage(deep=True))




Memory Usage Before:
Index           128
bike_name     11984
price          1192
city           9546
kms_driven     1192
owner         10144
age            1192
power          1192
brand          9517
dtype: int64

Memory Usage After:
Index          128
bike_name      447
price         1192
city           447
kms_driven    1192
owner          447
age           1192
power         1192
brand          447
dtype: int64


In [31]:
!pip install pymysql


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00


In [36]:
import pandas as pd
from sqlalchemy import create_engine

# Assuming 'data' is your DataFrame after the data type changes
#data = pd.read_csv('/content/used_bikes.csv')  # Replace with the actual file path

# Reset the index before flushing to MySQL(as requested in the task)
data.reset_index(drop=True, inplace=True)

# Replace 'username', 'password', 'host', and 'database' with your actual MySQL database connection details
# Im not hardcoding my own database details for security reasons, also a better approach to for increased security would be to use a configuration file
db_connection_info = {
    'username': 'abhay',
    'password': 'xyz',
    'host': 'localPC',
    'database': 'randomDB',
}

# Creating MySQL connection string
connection_string = f"mysql+pymysql://{db_connection_info['username']}:{db_connection_info['password']}@{db_connection_info['host']}/{db_connection_info['database']}"

# Connecting to MySQL database
engine = create_engine(connection_string)



In [ ]:
# Dump the data into the MySQL database
data.to_sql(name='used_bikes', con=engine, if_exists='replace', index=False)

# Index the column after data insertion
column_to_index = 'bike_name'
with engine.connect() as connection:
    connection.execute(f"CREATE INDEX idx_{column_to_index} ON used_bikes({column_to_index})")

# Print the DataFrame with the 'bike_name' column indexed
indexed_data = pd.read_sql_query('SELECT * FROM used_bikes', engine)
print("\nIndexed Data:")
print(indexed_data)
